In [14]:
%run ../utilities/web_utilities.ipynb
%run ../utilities/database_utilities.ipynb
%run ../utilities/encoder_utilities.ipynb
%run ../utilities/pd_utilities.ipynb

In [15]:
write_mode = 'replace'
target_table = 'whl_player_season_stats'

In [16]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../../')

import settings as st

In [17]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:
db_host = st.database['local']['host']
db_port = st.database['local']['port']
db_user = st.database['local']['user']
db_pass = st.database['local']['pass']
db_sys = st.database['local']['system']
db_db = st.database['local']['db']

In [19]:
key = "41b145a848f4bd67"

In [20]:
sql = "SELECT DISTINCT player_id FROM whl_team_roster_by_season WHERE player_id IS NOT NULL ORDER BY player_id ASC"
res = query_database_to_list(db_sys, db_user, db_pass, db_host, db_port, db_db, sql)

In [28]:
master_df = pd.DataFrame()

In [29]:
i = 1
for r in res:
    player_id = str(r['player_id']).replace(".0", "")
    
    print(i, player_id)
    
    url = "http://lscluster.hockeytech.com/feed/?feed=modulekit&view=player&key={0}&fmt=json&client_code=whl&lang=en&player_id={1}&category=seasonstats".format(key, player_id)
    
    json_data = return_json(url)
    
    player_df = pd.DataFrame()
    
    try:
        playoff_df = pd.DataFrame(json_data['SiteKit']['Player']['playoff'])
        playoff_df = playoff_df[playoff_df.shortname != "total"]
        playoff_df['season_type'] = 'Playoff'
        
        player_df = player_df.append(playoff_df)
    except:
        pass
    
    try:
        regular_df = pd.DataFrame(json_data['SiteKit']['Player']['regular'])
        regular_df = regular_df[regular_df.shortname != "total"]
        regular_df['season_type'] = 'Regular'

        player_df = player_df.append(regular_df)
    except:
        pass
    
    master_df = master_df.append(player_df)
    
    i+=1

1 21812
2 25862
3 25874
4 25886
5 25908
6 25918
7 25924
8 25931
9 25940
10 25945
11 25985
12 25988
13 26020
14 26033
15 26043
16 26058
17 26063
18 26066
19 26076
20 26084
21 26090
22 26100
23 26101
24 26109
25 26110
26 26115
27 26117
28 26130
29 26137
30 26138
31 26139
32 26150
33 26154
34 26159
35 26163
36 26166
37 26173
38 26177
39 26178
40 26179
41 26182
42 26193


KeyboardInterrupt: 

In [31]:
"""
Clean up Data
"""
master_df['max_start_date'] = master_df.max_start_date.apply(lambda x: clearDate(x))

"""
Fix Data Types
"""
master_df["active"] = pd.to_numeric(master_df["active"])
master_df["assists"] = pd.to_numeric(master_df["assists"])
master_df["career"] = pd.to_numeric(master_df["career"])
master_df["empty_net_goals"] = pd.to_numeric(master_df["empty_net_goals"])
master_df["faceoff_attempts"] = pd.to_numeric(master_df["faceoff_attempts"])
master_df["faceoff_pct"] = pd.to_numeric(master_df["faceoff_pct"])
master_df["faceoff_wins"] = pd.to_numeric(master_df["faceoff_wins"])
master_df["first_goals"] = pd.to_numeric(master_df["first_goals"])
master_df["game_tieing_goals"] = pd.to_numeric(master_df["game_tieing_goals"])
master_df["game_winning_goals"] = pd.to_numeric(master_df["game_winning_goals"])
master_df["games_played"] = pd.to_numeric(master_df["games_played"])
master_df["goals"] = pd.to_numeric(master_df["goals"])
master_df["hits"] = pd.to_numeric(master_df["hits"])
master_df["insurance_goals"] = pd.to_numeric(master_df["insurance_goals"])
master_df["jersey_number"] = pd.to_numeric(master_df["jersey_number"])
master_df["overtime_goals"] = pd.to_numeric(master_df["overtime_goals"])
master_df["penalty_minutes"] = pd.to_numeric(master_df["penalty_minutes"])
master_df["penalty_minutes_per_game"] = pd.to_numeric(master_df["penalty_minutes_per_game"])
master_df["playoff"] = pd.to_numeric(master_df["playoff"])
master_df["plus_minus"] = pd.to_numeric(master_df["plus_minus"])
master_df["points"] = pd.to_numeric(master_df["points"])
master_df["points_per_game"] = pd.to_numeric(master_df["points_per_game"])
master_df["power_play_assists"] = pd.to_numeric(master_df["power_play_assists"])
master_df["power_play_goals"] = pd.to_numeric(master_df["power_play_goals"])
master_df["season_id"] = pd.to_numeric(master_df["season_id"])
master_df["shooting_percentage"] = pd.to_numeric(master_df["shooting_percentage"])
master_df["shootout_attempts"] = pd.to_numeric(master_df["shootout_attempts"])
master_df["shootout_goals"] = pd.to_numeric(master_df["shootout_goals"])
master_df["shootout_percentage"] = pd.to_numeric(master_df["shootout_percentage"])
master_df["shootout_winning_goals"] = pd.to_numeric(master_df["shootout_winning_goals"])
master_df["short_handed_assists"] = pd.to_numeric(master_df["short_handed_assists"])
master_df["short_handed_goals"] = pd.to_numeric(master_df["short_handed_goals"])
master_df["shots"] = pd.to_numeric(master_df["shots"])
master_df["team_id"] = pd.to_numeric(master_df["team_id"])
master_df["unassisted_goals"] = pd.to_numeric(master_df["unassisted_goals"])

master_df["max_start_date"] = pd.to_datetime(master_df["max_start_date"])

In [32]:
master_df = master_df.reset_index(drop=True)

In [33]:
if write_mode == 'append':
    index = get_table_new_id(db_sys, db_user, db_pass, db_host, db_port, db_db, target_table, 'index')
    master_df.insert(0, 'index', master_df.index + index)
else:
    master_df.insert(0, 'index', master_df.index)

In [34]:
master_df

,index,active,assists,career,empty_net_goals,faceoff_attempts,faceoff_pct,faceoff_wins,first_goals,gaa,game_tieing_goals,game_winning_goals,games_played,goals,goals_against,goals_against_average,hits,insurance_goals,jersey_number,losses,max_start_date,minutes_played,ot,ot_losses,overtime_goals,penalty_minutes,penalty_minutes_per_game,playoff,plus_minus,points,points_per_game,power_play_assists,power_play_goals,savepct,saves,season_id,season_name,season_type,seconds_played,shooting_percentage,shootout_attempts,shootout_goals,shootout_goals_against,shootout_losses,shootout_percentage,shootout_saves,shootout_shots,shootout_winning_goals,short_handed_assists,short_handed_goals,shortname,shots,shots_against,shotspct,shutouts,sosavepct,team_city,team_code,team_id,team_name,team_nickname,ties,unassisted_goals,wins
0,0,1.0,4,1,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,5,1,NaN,NaN,0.0,0.0,7.0,NaN,2017-10-18,NaN,NaN,NaN,0.0,0,0.000000,1,4.0,5.0,1.00,0.0,0.0,NaN,NaN,263,2017 WHL Cup,Playoff,NaN,25.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,WHL Cup,4.0,NaN,NaN,NaN,NaN,British Columbia,BC,238,Team BC,Team British Columbia,NaN,0.0,NaN
1,1,1.0,1,1,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,20,3,NaN,NaN,0.0,0.0,18.0,NaN,1999-09-24,NaN,NaN,NaN,0.0,0,0.000000,0,-4.0,4.0,0.20,0.0,1.0,NaN,NaN,211,1999-2000 WHL Season,Regular,NaN,100.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1999-00 Seas,3.0,NaN,NaN,NaN,NaN,Lethbridge,LET,205,Lethbridge Hurricanes,Lethbridge,NaN,0.0,NaN
2,2,0.0,0,1,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,22,1,NaN,NaN,0.0,0.0,7.0,NaN,1998-09-25,NaN,NaN,NaN,0.0,0,0.000000,0,-4.0,1.0,0.05,0.0,0.0,NaN,NaN,209,1998-99 WHL Season,Regular,NaN,100.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1998-99 Seas,1.0,NaN,NaN,NaN,NaN,Lethbridge,LET,205,Lethbridge Hurricanes,Lethbridge,NaN,0.0,NaN
3,3,1.0,3,1,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,7,0,NaN,NaN,0.0,0.0,14.0,NaN,2016-03-21,NaN,NaN,NaN,0.0,13,1.857143,1,2.0,3.0,0.43,1.0,0.0,NaN,NaN,254,2016 WHL Playoffs,Playoff,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,2016Playoffs,0.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Blazers,NaN,0.0,NaN
4,4,1.0,10,1,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,15,1,NaN,NaN,0.0,0.0,14.0,NaN,2013-03-21,NaN,NaN,NaN,0.0,14,0.933333,1,0.0,11.0,0.73,2.0,1.0,NaN,NaN,243,2013 WHL Playoffs,Playoff,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,13 Playoffs,0.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Kamloops,NaN,0.0,NaN
5,5,1.0,4,1,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,11,3,NaN,NaN,0.0,0.0,14.0,NaN,2012-03-22,NaN,NaN,NaN,0.0,10,0.909091,1,1.0,7.0,0.64,0.0,0.0,NaN,NaN,240,2012 WHL Playoffs,Playoff,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0,12 Playoffs,0.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Kamloops,NaN,1.0,NaN
6,6,1.0,39,1,1.0,0.0,0.0,0.0,3.0,NaN,0.0,3.0,66,23,NaN,NaN,0.0,3.0,14.0,NaN,2015-09-24,NaN,NaN,NaN,0.0,55,0.833333,0,-8.0,62.0,0.94,16.0,6.0,NaN,NaN,251,2015-16 Regular Season,Regular,NaN,115.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,1.0,0.0,15-16 Reg,20.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Blazers,NaN,0.0,NaN
7,7,1.0,51,1,2.0,0.0,0.0,0.0,4.0,NaN,0.0,5.0,71,25,NaN,NaN,0.0,2.0,14.0,NaN,2014-09-18,NaN,NaN,NaN,0.0,63,0.887324,0,31.0,76.0,1.07,13.0,4.0,NaN,NaN,249,2014-15 Regular Season,Regular,NaN,250.0,2.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,1.0,0.0,14-15 Reg,10.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Kamloops,NaN,1.0,NaN
8,8,1.0,23,1,0.0,0.0,0.0,0.0,1.0,NaN,0.0,2.0,48,12,NaN,NaN,0.0,0.0,14.0,NaN,2013-09-18,NaN,NaN,NaN,0.0,37,0.770833,0,3.0,35.0,0.73,5.0,1.0,NaN,NaN,245,2013-14 Regular Season,Regular,NaN,0.0,2.0,1.0,NaN,NaN,50.0,NaN,NaN,0.0,1.0,1.0,13-14,0.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Kamloops,NaN,2.0,NaN
9,9,1.0,21,1,0.0,0.0,0.0,0.0,3.0,NaN,0.0,2.0,58,12,NaN,NaN,0.0,1.0,14.0,NaN,2012-09-19,NaN,NaN,NaN,0.0,50,0.862069,0,10.0,33.0,0.57,4.0,2.0,NaN,NaN,242,2012-13 Regular Season,Regular,NaN,0.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,1.0,1.0,2012-13,0.0,NaN,NaN,NaN,NaN,Kamloops,KAM,203,Kamloops Blazers,Kamloops,NaN,0.0,NaN


In [35]:
write_df_to_database(master_df, target_table, db_sys, db_user, db_pass, db_host, db_port, db_db, write_mode, False)